# Tutorial 1: Running global CTSM simulations 

This tutorial is an introduction to running global scale simulations of the Community Terrestrial Systems Model (CTSM).  It will guide you through running a simulation and provides example visualization of simulation results. Once you complete the global simulation in this tutorial, you can use the `Global_Visualization_Tutorial` to explore the model data further.

In the previous `NEON_Simulation_Tutorial` example many steps to run CLM were condensed into a single function. Here you'll how to create your own case, setup, build, and run the simulation. *These are the basic steps reqired to run CLM and modify the code to conduct model experiments*
<br><br>

### A few notes about the model:
- There are several configuration options of CTSM, and throughout this tutorial we will use the Community Land Model (CLM) configuration which is the climate mode of CTSM. Throughout the rest of this tutorial, we refer to the model as CLM and will use version 5.1 with satelity phenology (**CLM5.1-SP**).

- **Satelite phenology (or SP)** simulations prescribe he distribution of vegetation and its leaf area index (LAI). These SP simulations useful for understanding water and energy fluxes that are simulated by the model, as well as gross primary productivity (GPP). SP simulations focus on the *biogeophysics* of the model.  Accordingly, there is no *biogeochemistry* (i.e., carbon fluxes) in SP simulations downstream of GPP calculations. This simplification removes potential feedbacks or biases that may occur from plant allocation, autotrophic respiration, and turnover on simulated LAI. SP simulations can be conducted with CLM5.1-SP, or with CLM_FATES-SP.

- **FATES** - the Functionally Assembled Terrestrial Ecosystem Simulator - more relistically represents terrestrial vegetation by simulating forest size structure, disturbance, and competition.  FATES is an external module which can run within a given “Host Land Model”, here we are using CLM.  This examples uses the satelite phenology scheme in FATES, one of the reduced complexity modes of FATES that helps to isolate representation of canopy processes like radiative transfer and photosynthesis. Several FATES configurations are available in CLM.  
<br><br>

### Additional information:
Additional information about CTSM and CLM is available [on the website](https://www.cesm.ucar.edu/models/cesm2/land/), including [technical documentation](https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/index.html), a [user's guide](https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/index.html), and a [quickstart guide](https://escomp.github.io/CESM/release-cesm2/quickstart.html#create-a-case) for running various model configurations beyond what is covered in this tutorial.

Additional information about FATES is available [on the FATES github site](https://github.com/NGEET/fates). TODO ADD additional documentation here.
<br><br>
### Questions about this tutorial? 
- Please post them on the [CTSM forum in the CESM Bulletin Board](https://bb.cgd.ucar.edu/cesm/forums/ctsm-clm-mosart-rtm.134/). Note that this resource will require you to register and log in so that you can be notified of responses to your inquiries. 
- You can also file issues on the [NCAR CTSM-Tutorial GitHub repository](https://github.com/NCAR/CTSM-Tutorial-2022).
***

## In this tutorial

The tutorial has several components. Below you will find steps to: 
1. Run an out-of-the-box CLM simulation using CLM_FATES-SP
2. Locate model history files
3. Basic visualization of model history files

## Note This tutorial assumes that you've done your homework! 

If you haven't downloaded CTSM from the github repository you need to go back to the `NEON_Simulation_Tutorial` and do this first.
***

<h1> 1. Set up and run a simulation</h1>

**CTSM can be run in 4 simple steps.** 

<h3>1.1 create a new case </h3>

- *This step sets up a new simulation. It is the most complicated of these four steps because it involves making choices to set up the model configuration*

### 1.2 invoke case.setup
- *This step configures the model so that it can compile*

### 1.3 build the executable
- *This step compiles the model*

### 1.4 submit your run to the batch queue
- *This step submits the model simulation to the cloud*

***
<h1> 1.1 create a new case</h1>

*Set up a new simulation*
***


## 1.1.1 create a directory for cases
This is a one-time step to create a directory to store your experiment cases:
#### **To Do:** 
`mkdir ~/clm_tutorial_cases`

*Note:* all items marked with **To Do** should be done on the comand line.

## 1.1.2 navigate to the source code directory
TODO ADJUST FOR DIRECTORY STRUCTURE IN THE CLOUD
#### **To Do:** 
`cd /glade/u/home/$USER/ctsm5.1dev090/cime/scripts`

## 1.1.3 Create a new case
#### **To Do:** 
`!./create_newcase --case ~/clm_tutorial_cases/I2000_CTSM_FATESsp --res f45_g37 --compset I2000Clm51Fates  --run-unsupported` 

### **./create_newcase**
There is a lot of information that goes into creating a case.

You can learn more about the options by typing `./create_newcase --help` on the the command line.

We'll briefly go over some of the highlights here.

### Required argements to create a new case
There are 3 Required arguments Needed to create a new case.  These include 
1. `--case`, which specifies the *location* and *name* of the case being created
 - `~` = your home directory
 - `clm_tutorial_cases` = the subdirectory we created to store your cases in 1.1.1
 - `I2000_CTSM_FATESsp` = the name of the case you're creating
 - *Recommendation:* Use meaningful names, including model version, type of simulation, and any additional details to help you remember the configuration of this simulation
<br><br>  
2. `--res` Defines the model *resolution*, or grid,
 - `f45_g37`, which is an alias for a 4x5 degree grid on the atmosphere and land (*f45*).  This coarse resolution is more efficient for testing. Note, *g37* refers to the resolution of the ocean model. Don't worry the ocean model won't be used for this case.
 - commonly the land model is run at a nominal 1 degree `f09_g17` or 2 degree `f19_g17` resolution 
 - Using `./query_config --res` provides a list of supported model resolutions
<br><br>
3. `--compset` Defines the *component set* for your case, 
 - The Component set specifies the default configuration for the case which includes:
   - Component models (e.g. active vs. data vs. stub), 
   - Timpe period of simulations and forcing scenarios (e.g. 1850 vs 2000 vs. HIST) and 
   - Physics options (e.g. CLM5.1 vs CLM5.0). 
 - `I2000Clm51Fates` is alias that actually describes a much longer set of components that are being used for this case. 
 - All CLM-only compsets start with *"I"*.
 - Using `./query_config --compsets clm` provides examples of other CLM compsets

### More on component sets
The long name for the compset is `2000_DATM%GSWP3v1_CLM51%FATES_SICE_SOCN_MOSART_SGLC_SWAV`, which defines
   - time = `2000_` (vs. 1850, HIST, SSP, etc)
   - data atmosphere `DATM%GSWP3v1_`, here from GSWP3, as opposed to model atmosphere (CAM) 
   - land model `CLM51%FATES_` CLM5.1 physics package with FATES
   - stub sea ice model `SICE_`
   - stub ocean model `SOCN_` (see there's no active ocean model!)
   - mosart river model `MOSART_`
   - stub glacier `SGLC_`
   - stub wave model `SWAV_`   

<div>
<img src="../images/Components.png" width="460" height="460" />
</div>


Key Definitions:
- **Active:** Simulation is using the code from the model during the run
- **Data:** Simulation is reading in data from a file for this component
- **Stub:** Component is not being used
<br><br> 
*Note: Some compsets are scientifically supported and others are not*. You can use an unsupported compset, but will need to add the option `--run_unsupported` at the end of the create_newcase command line.
<br><br> 
***
More information on model [Configurations and Grids can be found on  the CESM website](https://www.cesm.ucar.edu/models/cesm2) (see *Configurations and Grids* subheading at the bottom of the page)

***
<h1> 1.2 Invoke case.setup </h1>

*This step configures the model so that it can compile* 
***


## 1.2.1 Move to your case directory
#### **To Do:** 
`cd ~/clm_tutorial_cases/I2000_CTSM_FATESsp`

## 1.2.2 Set up your case
#### **To Do:**
`./case.setup`

Using this command, we just configured the model and created the files to modify options & input data.
<div>
<img src="../images/Setup.png" width="670" />
</div>

***
<h1> 1.3 Build the executable  </h1>

*This step compiles the model*
***

#### **To Do:** 
`./case.build`

Note that `qcmd -- ./case.build` is specific for cheyenne and runs the command on a computing node, reducing the load on the login node. 

**You must include `qcmd --` when running on cheyenne.**

The `./case.build` script:
1. Checks input data
2. Creates a build/run directory with model executable and namelists

***
<h1> 1.4 Submit your run  </h1>

*This step submits the model simulation*

But first we need to check on a few thing
***

## Case Customization Checks
The model is now compiled and ready to run! 

There are a few things we should check before submitting the run. 

Customizing your case can happen in a few ways. Here we'll introduce:
1. XML change (`*.xml` files), and
2. Namelist modifications (`user_nl_*` files)

## 1.4.1 XLM files
You may want to customize a number of features for your simulation.  For example:
1. How many days or years will the model simulate?
2. How much time does the computer need for this simulation?
3. Which computing project account is the model charging to?

These options are specified in the `env_*.xml` files in your case directory

The XML files can be modified directly, but we recommend that you use the `xmlchange` script.
Next, we’ll review how to check and modify variables in XML files.

### `xmlchange`
Using the `xmlchange` script is the preferred method to modify XML files, but you can edit XML files by hand

Benefits of using `xmlchange`:
1. Allows changing variables in env_*.xml files using a command-line interface
2. Won’t let you mess up the syntax! The script checks the setting immediately for validity.
3. Settings are copied into the CaseStatus file, providing documentation of your changes.

<div>
<img src="../images/xmlchange.png" width="670" />
</div>

The `env_batch.xml` and `env_run.xml` files include most of the variables you will need to modify
to set up and run simulations.

These files can be changed at any time before running the simulation.
<br><br> 
### A few useful tips for using the xml scripts:
1. Use `./xmlquery --listall` to list variables and their values in the .xml files
2. Modify a variable in a .xml file, use `./xmlchange`
3. For help, type `./xmlchange --help`

#### Example: 
`./xmlchange {variable to be changed}={value to change to}`
<br><br> 

*****

Many runtime variables are found in the `env_run.xml` file. The variables in this file control the mechanics of the run (length, resubmits, and archiving).

Common variables in env_run.xml to change include:
1. `STOP_OPTION` sets the run-time interval type, i.e. nmonths, ndays, nyears
2. `STOP_N` sets the number of run-time intervals to run the model during the specified wallclock time.
 - Wallclock time is set in the env_batch.xml file and is a measure of the actual time. 
3. `RESUBMIT` sets the number of times to resubmit the run

#### **To Do:** 
Use `xmlquery` to find the values of the variables listed above. Feel free to play around with options 

`./xmlquery STOP_OPTION`

*Note:* Use `./xmlquery --listall` to find the values of all variables in all XML files, or `./xmlquery --file <env_run.xml>` to list variables in a specific file (here env_run.xml). You can also search for multiple variables, separating the variable names with a comma (“,”). You can also search for strings using `./xmlquery –p <string>` . This will search for every variable that includes this string. Try it for “STOP” or “CLM”.
    
*****
   
From the list above, you should find these values:
1. `STOP_OPTION` -> ndays
2. `STOP_N` -> 5
3. `RESUBMIT` -> 0
*Note:* If you try to change a variable to a value that isn’t an option, you will get an error message with a list of valid values.

- By default, the model is set to run for 5 days. 

*We will not see any model output from a 5-day model run because history files with model output are only recorded every month by default*

- Let’s change the length of the simulation to 1 year.

### **To Do:**
`./xmlchange STOP_OPTION=nyears`

`./xmlchange STOP_N=1`

This changes the run-time interval from days to years. 
Verify that the changes worked as you expected:

`./xmlquery STOP_OPTION`

`./xmlquery STOP_N`

*Note: If you try to change a variable to a value that isn’t an option, you will get an error message with a list of valid values.*
<br><br> 

*****

We also need to check that we are using the correct project code and giving the computer enough time to run the simulation. These are found in the `env_batch.xml` file.
### **To Do:**
*Use xmlquery to check*
1. PROJECT
2. JOB_WALLCLOCK_TIME

Do you remember how to do this? If not, review the command-line prompts above.


***

It won’t take 12 hours to run a 5-year simulation and your simulation will get into the queue more quickly if you set a shorter run time.

A typical CLM-only 1850 satellite phenology (SP) simulation at 2 degree resolution will run ~115 years in 12 hours.

*How long will it take to run 5 years?* (I usually round up to the nearest 30 minute interval)

CLM-FATESsp is more slower to run than CLM-SP.  We're also running with a limited number of nodes on the cloud.
4x5 resolution CLM-FATESsp cases like the one we have here run ~85 years in 12 hours.
### **To Do:**
Change the JOB_WALLCLOCK_TIME to the amount of time you estimate your simulation will take (the answer to the above question)

Do you remember how to do this? If not, review the command-line prompts above.

*Note:* You can find out timing information for some standard simulations [here](https://csegweb.cgd.ucar.edu/timing/cgi-bin/timings.cgi). Click on a compset similar to the one you will run and look for the “model throughput” value. This is how many simulated years the computer can run in a 24-hour timeframe.  *If you're running on cheyenne*, you can estimate the amount of time you’ll need based on this.



## 1.4.2 Namelist changes
You may want to customize additional features for your simulation. These can be controled with namelist changes.
- Not all changes can be made with `./xmlchange`
- Additional changes made using namelist files: `user_nl_<model>`
- `user_nl_<model>` files are created in the case directory after `./case.setup`

The compset that you created your case with sets up initial, or default namelist options. These can be found in `CaseDocs/lnd_in`
- `user_nl_clm` modifies the namelist `lnd_in` options
- *Important:* Don’t modify the namelist file (`lnd_in`) directly. Instead, make changes in `user_nl_clm`.
- Some namelist variables can also be changed in env_run.xml file. 
- This [website has CLM5.0 namelist variables](http://www.cesm.ucar.edu/models/cesm2/settings/current/clm5_0_nml.html)

#### **To Do:** 
Turn on the SP configuration of FATES

`echo "use_fates_sp = .true." >> user_nl_clm`

This is a simplified way of writing information to `user_nl_clm`

Check that it worked by opening the file in an editor

#### **To Do:** 
`emacs user_nl_clm`

_or_  
 
`vi user_nl_clm`

**Is fates_sp set to true?**

## 1.4.3 Submit the case!

#### **To Do**
`./case.submit`

When you submit a job, you will see confirmation that it successfully submitted:

### Congratulations! You've created and run CLM for the NEON tower you selected.

Next, you will probably want to check on the status of your jobs:
#### **To Do**
`qstat -u $USER`


If you want to stop the simulation, you will need to kill your job.
- Find your Job ID after typing `qstat`
- Type ‘qdel <Job ID> ‘



TODO
[See slides](https://www.cgd.ucar.edu/events/2019/ctsm/files/practical1-lombardozzi.pdf)

****
# 2. Locate model history files

# 3. Basic visualization of model history files
****